In [ ]:
!pip install degiro_connector pandas scipy sqlitedict polars yfinance
#!pip install selenium
!pip install --upgrade lxml


In [ ]:
from screener import Screener 
import logging
import sys
import pandas as pd

pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

logfile = "./outdegiro.log"
logging_level = logging.INFO
logging.basicConfig(
    level=logging_level,
    handlers=[logging.FileHandler(logfile, mode="a"), logging.StreamHandler(sys.stdout), ],
    format="%(asctime)s - %(name)s:%(filename)s:%(funcName)s:%(lineno)d - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

'''
logging.getLogger("yfinance").setLevel(logging.DEBUG)
logging.getLogger("requests_cache").setLevel(logging.DEBUG)
logging.getLogger("cachedYahooApi").setLevel(logging.DEBUG)

main_logger = logging.getLogger()
'''

df = Screener(None, None, None, None)


In [ ]:

if df is not None:
    QS = 80.0  # score loic
    QSP = 95.0  # score loic + perf
    REV = 5   # croissance revenu
    MRG = 10   # marge
    SLD = 0  # en solde de x%
    LH = 1  # cours relatif entre le plus bas annuel et le plus haut [0,1]
    YLD = 10  # rendement dividende+prix
    PRX = 0   # croissance annuelle du prix de l'action, sans les dividendes
    ddf = df.copy()
    print(list(df.columns))
     
    ddf = ddf[
        (ddf["qscore"] >= QS)
        & (ddf["qscorePerf"] >= QSP)
        & (ddf["REVPS5YGR"] >= REV)
        & (ddf["MARGIN5YR"] >= MRG)
        & (ddf["En Solde"] >= SLD)
        & (ddf["L%H"] <= LH)
        & (ddf["ChPctPrice5Y"] >= PRX)
        & (ddf["YLD+PRY"] >= YLD)
    
    ]
    
    ddf = ddf.sort_values("scorePerf", ascending=False)
    
    ddf = ddf[
        [
            "VOL10DAVG","MKTCAP.USD","MKTCAP", "YSymbol", "isin", "name", "sector", "industry", "country", "score", "qscore", "scorePerf", "qscorePerf", "MKTCAP.USD","qMKTCAP.USD", "MKTCAP","priceCurrency", "REVPS5YGR", "MARGIN5YR", "PER", "Rendement", "En Solde", "L%H", "ChPctPrice5Y", "AROE5YAVG", "closePriceDate",
        ]
    ]
    
    print(ddf.shape)
    display(ddf)
    
    #ddf.to_csv("extrait.csv", index=False, sep=str(";"), decimal=str(","), encoding="utf-8-sig", float_format="%.3f", quoting=csv.QUOTE_MINIMAL)
